# Code to extract the data from the  ffc website

## Import the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import lxml 

In [2]:
# need to manually save the file as csv from the website : http://www.f2concept.com/cal_aura/FR/

# load data from csv file, the first row is the header
df = pd.read_csv('Calendrier_aura.csv', sep=';', header=0, encoding='ISO-8859-1')

In [3]:
df['Catégorie'].unique()

array(['BMX Racing', 'Toutes catégories', 'Américaine', 'Cyclosportive',
       'Ecole de VTT', 'Gentlemen', 'Randonnée', 'Grimpée chronométrée',
       'Test chronométré', 'Ecole de route', 'Trial Toutes catégories',
       'XC Toutes catégories', 'Access hommes', 'U17 hommes',
       'Open1 hommes et +', 'Elite hommes (régional) et +', 'VAE',
       'DH Toutes catégories', 'Vétathlon', 'Open2 hommes et +',
       'Elite femmes (régional) et +', 'U15+U17 femmes', 'U15 hommes',
       'Elite nationale hommes', 'Open3 hommes et +',
       'U19 hommes fédéral', 'Enduro', 'UCI Femmes', 'Chronométré',
       'U15 femmes', 'U17 femmes', 'U19 hommes régional', 'UCI U23',
       'UCI classe 1 et World Tour', 'UCI Juniors', 'Ultra distance',
       'XC U15-U17', 'XC Marathon', 'Cyclisme artistique',
       'Non Chronométré', 'UCI classe 2', 'Access femmes',
       'Elite nationale femmes'], dtype=object)

In [4]:
# change column names lieu to Lieu 
df.rename(columns={'lieu': 'Lieu'}, inplace=True)

# Keep only the following Catégoie : [Gentlemen, Grimpée chronométrée, Test chronométré, Access hommes]
courses_ffc = df[df['Catégorie'].isin(['Gentlemen', 'Grimpée chronométrée', 'Test chronométré', 'Access hommes', 'Cyclosportive'])]
courses_ffc['Du'] = courses_ffc['Du'].astype(str).str.replace('/23', '/2023')
courses_ffc['Au'] = courses_ffc['Au'].astype(str).str.replace('/23', '/2023')
courses_ffc.head(5)




C:\Users\theop\AppData\Local\Temp\ipykernel_22100\1637306300.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses_ffc['Du'] = courses_ffc['Du'].astype(str).str.replace('/23', '/2023')
C:\Users\theop\AppData\Local\Temp\ipykernel_22100\1637306300.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses_ffc['Au'] = courses_ffc['Au'].astype(str).str.replace('/23', '/2023')


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
8,15/10/2023,15/10/2023,CYCLO LES HÉROS,Cyclosportive,ALPES VELO,Route,1,SAINT-VULBAS
13,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER
17,14/10/2023,14/10/2023,TROPHEE DES GRIMPEURS MONTEE CLAUDE SEGUY,Grimpée chronométrée,VELO SPORT BRIVADOIS,Route,43,SAINT CIRGUES-LAVOUTE CHILHAC
18,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
19,14/10/2023,14/10/2023,CHALLENGE DES MURES,Test chronométré,E.C. MUROISE,Route,69,SAINT LAURENT DE MURE


In [5]:
# Show the number of courses by region
courses_ffc['Département'].value_counts()

74    27
1     22
38    16
63    12
15    10
73     9
42     8
69     6
43     5
26     4
7      3
3      1
Name: Département, dtype: int64

## Extract valuable informations

In [6]:
# Split the datefram into 4 dataframes, one for each category
gentlemen = courses_ffc[courses_ffc['Catégorie'] == 'Gentlemen']
grimpée = courses_ffc[courses_ffc['Catégorie'] == 'Grimpée chronométrée']
chrono = courses_ffc[courses_ffc['Catégorie'] == 'Test chronométré']
access = courses_ffc[courses_ffc['Catégorie'] == 'Access hommes']

# save the number of rows for each dataframe
gentlemen_rows = gentlemen.shape[0]
grimpée_rows = grimpée.shape[0]
chrono_rows = chrono.shape[0]
access_rows = access.shape[0]



# change the date format to datetime
gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')
grimpée['Du'] = pd.to_datetime(grimpée['Du'], format='%d/%m/%Y')
chrono['Du'] = pd.to_datetime(chrono['Du'], format='%d/%m/%Y')
access['Du'] = pd.to_datetime(access['Du'], format='%d/%m/%Y')


# Sort the dataframe by date
gentlemen = gentlemen.sort_values(by=['Du'])
grimpée = grimpée.sort_values(by=['Du'])
chrono = chrono.sort_values(by=['Du'])
access = access.sort_values(by=['Du'])

# reverse the date y/m/d to d/m/y
gentlemen['Du'] = gentlemen['Du'].dt.strftime('%d/%m/%Y')
grimpée['Du'] = grimpée['Du'].dt.strftime('%d/%m/%Y')
chrono['Du'] = chrono['Du'].dt.strftime('%d/%m/%Y')
access['Du'] = access['Du'].dt.strftime('%d/%m/%Y')


# Check if there is a loss of data: the number of rows should be the same
print('gentlemen : ', gentlemen.shape[0] == gentlemen_rows)
print('grimpée : ', grimpée.shape[0] == grimpée_rows)
print('chrono : ', chrono.shape[0] == chrono_rows)
print('access : ', access.shape[0] == access_rows)



gentlemen :  True
grimpée :  True
chrono :  True
access :  True


C:\Users\theop\AppData\Local\Temp\ipykernel_22100\390231604.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')
C:\Users\theop\AppData\Local\Temp\ipykernel_22100\390231604.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grimpée['Du'] = pd.to_datetime(grimpée['Du'], format='%d/%m/%Y')
C:\Users\theop\AppData\Local\Temp\ipykernel_22100\390231604.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [7]:
access.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
744,26/02/2023,26/02/2023,PROLOGUE VALLEE DU BEDAT,Access hommes,VELO SPORT GERZATOIS,Route,63,BLANZAT
733,05/03/2023,05/03/2023,PRIX AUTO PIECES AUVERGNE,Access hommes,V C COURNON D'AUVERGNE,Route,63,COURNON D'AUVERGNE
722,11/03/2023,11/03/2023,PRIX TARARE POPEY,Access hommes,A.C. TARARE/POPEY,Route,69,SAINT ROMAIN DE POPEY
718,12/03/2023,12/03/2023,PRIX DE LA ZONE INDUSTRIELLE DE CUSSET,Access hommes,AVENIR CYCLISTE CUSSET,Route,3,CUSSET
705,12/03/2023,12/03/2023,2EME PRIX DE SAINT MAURICE DE SAINT REMENS,Access hommes,VELO CLUB D'AMBERIEU,Route,1,SAINT MAURICE DE REMENS


In [8]:
gentlemen.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
48,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V.C. PAYS DE ST FLOUR,Route,15,PAULHAC / VALUÉJOLS
23,07/10/2023,07/10/2023,GENTLEMEN DE TAPONAS,Gentlemen,U.C. BELLEVILLOISE,Route,69,BELLEVILLE
18,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
13,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER


In [9]:
grimpée.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
592,23/04/2023,23/04/2023,MONTEE CHRONOMETREE DE SIRAN,Grimpée chronométrée,A C V AURILLACOIS,Route,15,GLENAT
516,13/05/2023,13/05/2023,TROPHEE DES GRIMPEURS MONTEE SANCY OUEST,Grimpée chronométrée,V C DE BESSE,Route,63,LA TOUR D AUVERGNE
473,27/05/2023,27/05/2023,TROPHEE DES GRIMPEES CYCLISTES DE HAUTE SAVOIE...,Grimpée chronométrée,V.C. DOUVAINE,Route,74,BONS EN CHABLAIS
470,27/05/2023,27/05/2023,TROPHEE DES GRIMPEURS MONTEE AGGLO TOUR ISSOIR...,Grimpée chronométrée,ISSOIRE SPORT ORGANISATION,Route,63,ARDE/COUZE
435,03/06/2023,03/06/2023,TROPHEE DES GRIMPEURS MONTEE DE LA CROIX ROBERT,Grimpée chronométrée,TEAM CHATEL GUYON,Route,63,LAC CHAMBON


In [10]:
chrono.head(5)


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
635,15/04/2023,15/04/2023,CHAMPIONNATS AUVERGNE-RHÔNE-ALPES ROUTE CLM PA...,Test chronométré,TEAM CYCLISTE CHATEL-GUYON,Route,63,CHATEL GUYON
368,11/06/2023,11/06/2023,MONTEE VELOCIO,Test chronométré,E.C. ST ETIENNE - LOIRE,Route,42,SAINT ETIENNE
19,14/10/2023,14/10/2023,CHALLENGE DES MURES,Test chronométré,E.C. MUROISE,Route,69,SAINT LAURENT DE MURE


In [11]:
import dataframe_image as dfi
#Save the dataframes as an image
dfi.export(gentlemen, 'gentlemen.png', table_conversion= 'matplotlib')
dfi.export(grimpée, 'grimpée.png', table_conversion= 'matplotlib')
dfi.export(chrono, 'chrono.png', table_conversion= 'matplotlib')
dfi.export(access, 'access.png', table_conversion= 'matplotlib')




In [12]:
# change the date format to datetime
gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')


# Sort the dataframe by date
gentlemen = gentlemen.sort_values(by=['Du'])

# reverse the date y/m/d to d/m/y
gentlemen['Du'] = gentlemen['Du'].dt.strftime('%d/%m/%Y')
gentlemen.head(5)




,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
48,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V.C. PAYS DE ST FLOUR,Route,15,PAULHAC / VALUÉJOLS
23,07/10/2023,07/10/2023,GENTLEMEN DE TAPONAS,Gentlemen,U.C. BELLEVILLOISE,Route,69,BELLEVILLE
18,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
13,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER


In [13]:
pd.set_option('display.max_rows', None)
chrono.query('`Département` == 74')


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu


## Create a map

In [14]:
# location
import geocoder
import geopy
from geopy.geocoders import Nominatim
from functools import partial

# map 
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

from tqdm import tqdm
# remove the warning
import warnings
warnings.filterwarnings('ignore')



In [15]:
# Use the initial dataframe :
courses_map = courses_ffc.copy()

# add the localisation for the "Etape du Tour ", "Echappe du col de la Loze"
courses_map.loc[courses_map['Nom de la course'] == 'ETAPE DU TOUR ANNEMASSE-MORZINE', 'Lieu'] = 'Annemasse'
courses_map.loc[courses_map['Nom de la course'] == 'ECHAPPE DU COL DE LA LOZE', 'Lieu'] = 'Méribel'


courses_map['info_place'] = 'Auvergne-Rhône-Alpes'  + ', ' + courses_map['Département'].astype(str) + ', ' + courses_map['Lieu']
courses_map['info_course'] = courses_map['Catégorie'] + ' - ' + courses_map['Nom de la course'] + ' - ' + courses_map['Du']


# for "RANDO DU FREJUS", change the 'info_place' to "BARDONECHE, FRANCE"
courses_map.loc[courses_map['Nom de la course'] == 'RANDO DU FREJUS', 'info_place'] = 'BARDONECHE, FRANCE'


# find localisation 
geolocator = Nominatim(user_agent="courses_ffc")
tqdm.pandas()
courses_map['Location'] = courses_map['info_place'].progress_apply(partial(geolocator.geocode, timeout=5, language='fr'))
courses_map['Point'] = courses_map['Location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

# verification
print('failed to geocode for', courses_map['Point'].isnull().sum(), 'race(s)')

# extract null rows and apply geocode again
null_rows = courses_map[courses_map['Point'].isnull()]
null_rows['info_place'] = 'Auvergne-Rhône-Alpes' + ', ' + null_rows['Lieu']
# for "RANDO DU FREJUS", change the 'info_place' to "BARDONECHE, FRANCE" ///avoid geolocator error
null_rows.loc[null_rows['Nom de la course'] == 'RANDO DU FREJUS', 'info_place'] = 'BARDONECHE, FRANCE'
tqdm.pandas()
null_rows['Location'] = null_rows['info_place'].progress_apply(partial(geolocator.geocode, timeout=5, language='fr'))
null_rows['Point'] = null_rows['Location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

# verification
print('failed to geocode again for', null_rows['Point'].isnull().sum(), 'race(s)')
# show race with no location :
null_rows[null_rows['Point'].isnull()].head(20)


100%|██████████| 123/123 [00:00<?, ?it/s]


failed to geocode for 15 race(s)


100%|██████████| 15/15 [00:00<00:00, 1868.34it/s]

failed to geocode again for 14 race(s)


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu,info_place,info_course,Location,Point
17,14/10/2023,14/10/2023,TROPHEE DES GRIMPEURS MONTEE CLAUDE SEGUY,Grimpée chronométrée,VELO SPORT BRIVADOIS,Route,43,SAINT CIRGUES-LAVOUTE CHILHAC,"Auvergne-Rhône-Alpes, SAINT CIRGUES-LAVOUTE CH...",Grimpée chronométrée - TROPHEE DES GRIMPEURS M...,None,None
48,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V.C. PAYS DE ST FLOUR,Route,15,PAULHAC / VALUÉJOLS,"Auvergne-Rhône-Alpes, PAULHAC / VALUÉJOLS",Gentlemen - GENTLEMEN - 24/09/2023,None,None
214,30/07/2023,30/07/2023,TROPHEE DES GRIMPEES CYCLISTES DE HAUTE SAVOIE...,Grimpée chronométrée,Comité départemental Haute-Savoie FFC,Route,74,MIEUSSY HAUTE SAVOIE,"Auvergne-Rhône-Alpes, MIEUSSY HAUTE SAVOIE",Grimpée chronométrée - TROPHEE DES GRIMPEES CY...,None,None
226,23/07/2023,23/07/2023,LES HAUTS DU RUESSIUM,Access hommes,UNION CYCLISTE LE PUY EN VELAY,Route,43,BLANZAC / SAINT PAULIEN,"Auvergne-Rhône-Alpes, BLANZAC / SAINT PAULIEN",Access hommes - LES HAUTS DU RUESSIUM - 23/07/...,None,None
252,15/07/2023,15/07/2023,L'AINDINOISE,Cyclosportive,ALPES VELO,Route,1,PARC DES OISEAUX > GRAND-COLOMBIER,"Auvergne-Rhône-Alpes, PARC DES OISEAUX > GRAND...",Cyclosportive - L'AINDINOISE - 15/07/2023,None,None
260,12/07/2023,13/07/2023,RANDO DU FREJUS,Cyclosportive,CENTRE FORMATION POLE PERFORMANCE ESPOIRS DU S...,Route,73,BARDONECHE,"BARDONECHE, FRANCE",Cyclosportive - RANDO DU FREJUS - 12/07/2023,None,None
263,09/07/2023,09/07/2023,LA MAURIENNE CLASSIQUE CYCLO,Cyclosportive,CHAMBERY CYCLISME ORGANISATION,Route,73,SAINT JULIEN MONT DENIS CROIX DE FER,"Auvergne-Rhône-Alpes, SAINT JULIEN MONT DENIS ...",Cyclosportive - LA MAURIENNE CLASSIQUE CYCLO -...,None,None
268,08/07/2023,08/07/2023,GRIMPEE DE CHAMROUSSE,Grimpée chronométrée,G.M.C. 38 (EYBENS FORMATION),Route,38,URIAGE LES BAINS - CHAMROUSSE,"Auvergne-Rhône-Alpes, URIAGE LES BAINS - CHAMR...",Grimpée chronométrée - GRIMPEE DE CHAMROUSSE -...,None,None
292,02/07/2023,02/07/2023,JPP NEUF DE COEUR,Cyclosportive,V.C. CLUSES SCIONZIER,Route,74,CLUSES LES CARROZ,"Auvergne-Rhône-Alpes, CLUSES LES CARROZ",Cyclosportive - JPP NEUF DE COEUR - 02/07/2023,None,None
355,18/06/2023,18/06/2023,PRIX DES 4 COMMUNES DE LA COMBE DU VAL,Access hommes,E.C. ARBENT-MARCHON,Route,1,VIEUX D IZENAVE,"Auvergne-Rhône-Alpes, VIEUX D IZENAVE",Access hommes - PRIX DES 4 COMMUNES DE LA COM...,None,None


In [17]:
# add manually the location for the problematic rows :
no_location = null_rows[null_rows['Point'].isnull()]
no_location.loc[no_location['Lieu'] == 'SAINT CIRGUES-LAVOUTE CHILHAC', 'Lieu'] = 'LAVOUTE CHILHAC'
no_location.loc[no_location['Lieu'] == 'PAULHAC / VALUÉJOLS', 'Lieu'] = 'VALUÉJOLS'
no_location.loc[no_location['Lieu'] == 'MIEUSSY  HAUTE SAVOIE', 'Lieu'] = 'MIEUSSY'
no_location.loc[no_location['Lieu'] == 'BLANZAC / SAINT PAULIEN', 'Lieu'] = 'BLANZAC'
no_location.loc[no_location['Lieu'] == 'PARC DES OISEAUX > GRAND-COLOMBIER ', 'Lieu'] = 'CULOZ'
no_location.loc[no_location['Lieu'] == 'SAINT JULIEN MONT DENIS CROIX DE FER', 'Lieu'] = 'SAINT-JULIEN-MONT-DENIS'
no_location.loc[no_location['Lieu'] == 'URIAGE LES BAINS - CHAMROUSSE', 'Lieu'] = 'URIAGE-LES-BAINS'
no_location.loc[no_location['Lieu'] == 'CLUSES LES CARROZ', 'Lieu'] = 'CLUSES'
no_location.loc[no_location['Lieu'] == 'VIEUX D IZENAVE', 'Lieu'] = 'IZENAVE'
no_location.loc[no_location['Lieu'] == 'BELLEFONTAINE, ROUTE DE LA CHAPELLE DE LA TOUR', 'Lieu'] = 'SAINT CLAIR DE LA TOUR'
no_location.loc[no_location['Lieu'] == 'ARDE/COUZE', 'Lieu'] = 'ARDE'
no_location.loc[no_location['Lieu'] == 'AUTOUR DU LEMAN', 'Lieu'] = 'EVIAN-LES-BAINS'
no_location.loc[no_location['Lieu'] == 'VARENNE SAINT SAUVEUR/EXTERIEUR', 'Lieu'] = 'BOURG EN BRESSE'
# delete the problematic rows :
no_location = no_location[no_location['Lieu'] != 'BARDONECHE']  #CycloRando du Frejus, pas intéressante de toute facon

no_location['info_place'] = 'Auvergne-Rhône-Alpes' + ', ' + no_location['Département'].astype('str') + ', ' + no_location['Lieu']
tqdm.pandas()
no_location['Location'] = no_location['info_place'].progress_apply(partial(geolocator.geocode, timeout=10, language='fr'))
no_location['Point'] = no_location['Location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

# verification
print('failed to geocode again for', no_location['Point'].isnull().sum(), 'race(s)')
# show race with no location :
no_location[no_location['Point'].isnull()].head(20)



100%|██████████| 14/14 [00:00<?, ?it/s]

failed to geocode again for 1 race(s)


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu,info_place,info_course,Location,Point
260,12/07/2023,13/07/2023,RANDO DU FREJUS,Cyclosportive,CENTRE FORMATION POLE PERFORMANCE ESPOIRS DU S...,Route,73,BARDONECHE,"Auvergne-Rhône-Alpes, 73, BARDONECHE",Cyclosportive - RANDO DU FREJUS - 12/07/2023,None,None


In [18]:
# replace the null rows with the new values
null_rows.loc[null_rows['Point'].isnull(), 'Point'] = no_location['Point']
null_rows.loc[null_rows['Location'].isnull(), 'Location'] = no_location['Location']

# replace the null rows with the new values
courses_map.loc[courses_map['Point'].isnull(), 'Point'] = null_rows['Point']
courses_map.loc[courses_map['Location'].isnull(), 'Location'] = null_rows['Location']

# delete null rows
courses_map = courses_map[courses_map['Point'].notnull()] 

# add the latitude and longitude to the dataframe
courses_map[['Latitude', 'Longitude', 'Alt']] = pd.DataFrame(courses_map['Point'].tolist(), index=courses_map.index)
# delete Alt column
del courses_map['Alt']
courses_map.head(2)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu,info_place,info_course,Location,Point,Latitude,Longitude
8,15/10/2023,15/10/2023,CYCLO LES HÉROS,Cyclosportive,ALPES VELO,Route,1,SAINT-VULBAS,"Auvergne-Rhône-Alpes, 1, SAINT-VULBAS",Cyclosportive - CYCLO LES HÉROS - 15/10/2023,"(Saint-Vulbas, Belley, Ain, Auvergne-Rhône-Alp...","(45.8306, 5.28959, 0.0)",45.830600,5.289590
13,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER,"Auvergne-Rhône-Alpes, 74, SCIENTRIER",Gentlemen - GENTLEMAN DE SCIENTRIER - 15/10/2023,"(Scientrier, Saint-Julien-en-Genevois, Haute-S...","(46.1188775, 6.3163228, 0.0)",46.118878,6.316323


In [19]:
# drop null rows
courses_map = courses_map[courses_map['Point'].notnull()]

In [20]:
# slightly change the latitude and longitude to avoid overlapping of the markers
courses_map['Latitude'] = courses_map['Latitude'] + np.random.uniform(-0.01, 0.01, courses_map.shape[0])
courses_map['Longitude'] = courses_map['Longitude'] + np.random.uniform(-0.01, 0.01, courses_map.shape[0])


In [21]:
# create a map with the courses location and marker color by category
map_courses = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')



##### MARKERS #####
# color by category : blue,       green,    orange,   purple,       red : 
#                   Gentlemen,   Grimpée,   Chrono,   Access,   Cyclosportive


# define dataframes by category :
gentlemen = courses_map[courses_map['Catégorie'] == 'Gentlemen']
grimpée = courses_map[courses_map['Catégorie'] == 'Grimpée chronométrée']
test = courses_map[courses_map['Catégorie'] == 'Test chronométré']
access = courses_map[courses_map['Catégorie'] == 'Access hommes']
cyclosportive = courses_map[courses_map['Catégorie'] == 'Cyclosportive']


##### LAYERS #####
# each layer corespond to a category
access_group = folium.FeatureGroup(name="Access").add_to(map_courses)
for i in range(0,len(access)):
    access_group.add_child(folium.Marker([access.iloc[i]['Latitude'], access.iloc[i]['Longitude']], popup=access.iloc[i]['info_course'], icon=folium.Icon(color='purple', prefix='fa', icon='bicycle')))
gentlemen_group = folium.FeatureGroup(name="Gentlemen").add_to(map_courses)
for i in range(0,len(gentlemen)):
    gentlemen_group.add_child(folium.Marker([gentlemen.iloc[i]['Latitude'], gentlemen.iloc[i]['Longitude']], popup=gentlemen.iloc[i]['info_course'], icon=folium.Icon(color='blue', prefix='fa', icon='bicycle')))
grimpée_group = folium.FeatureGroup(name="Grimpée").add_to(map_courses)
for i in range(0,len(grimpée)):
    grimpée_group.add_child(folium.Marker([grimpée.iloc[i]['Latitude'], grimpée.iloc[i]['Longitude']], popup=grimpée.iloc[i]['info_course'], icon=folium.Icon(color='green', prefix='fa', icon='bicycle')))
test_group = folium.FeatureGroup(name="Test chronométré").add_to(map_courses)
for i in range(0,len(test)):
    test_group.add_child(folium.Marker([test.iloc[i]['Latitude'], test.iloc[i]['Longitude']], popup=test.iloc[i]['info_course'], icon=folium.Icon(color='orange', prefix='fa', icon='bicycle')))
cyclosportive_group = folium.FeatureGroup(name="Cyclosportive").add_to(map_courses)
for i in range(0,len(cyclosportive)):
    cyclosportive_group.add_child(folium.Marker([cyclosportive.iloc[i]['Latitude'], cyclosportive.iloc[i]['Longitude']], popup=cyclosportive.iloc[i]['info_course'], icon=folium.Icon(color='red', prefix='fa', icon='bicycle')))

# add button to switch between OPENSTREETMAP and STAMEN TERRAIN
folium.TileLayer('OpenStreetMap').add_to(map_courses)
folium.TileLayer('Stamen Terrain').add_to(map_courses)
# add a layer control
folium.LayerControl().add_to(map_courses)




##### LEGEND #####
# add a legend outside the map
legend_html =   '''
                <div style="position: fixed;
                            background-color:white; opacity:0.75;
                            bottom: 20px; right: 20px; width: 180px; height: 200px;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Légende : <br>
                                &nbsp; Gentlemen &nbsp; <i class="fa fa-bicycle fa-2x" style="color:blue"></i><br>
                                &nbsp; Grimpée &nbsp; <i class="fa fa-bicycle fa-2x" style="color:green"></i><br>
                                &nbsp; Test chronométré &nbsp; <i class="fa fa-bicycle fa-2x" style="color:orange"></i><br>
                                &nbsp; Access &nbsp; <i class="fa fa-bicycle fa-2x" style="color:purple"></i><br>
                                &nbsp; Cyclosportive &nbsp; <i class="fa fa-bicycle fa-2x" style="color:red"></i>
                </div>
                '''
# decrease the size of the legend
legend_html = legend_html.replace('font-size:1px', 'font-size:6px')
# write 'Légende' in bold
legend_html = legend_html.replace('Légende :', '<b>Légende :</b>')
map_courses.get_root().html.add_child(folium.Element(legend_html))







# save the map
map_courses.save('map_courses.html')

map_courses


In [22]:
# create a map with the courses location and marker color by category for each month
map_courses_month = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')

# convert the 'Au' column to datetime format
courses_map['Au'] = pd.to_datetime(courses_map['Au'], format='%d/%m/%Y')
# add the 'Mois' column to the dataframe
courses_map['Mois'] = courses_map['Au'].dt.month_name()


# define dataframes by month :
mars = courses_map[courses_map['Mois'] == 'March']
avril = courses_map[courses_map['Mois'] == 'April']
mai = courses_map[courses_map['Mois'] == 'May']
juin = courses_map[courses_map['Mois'] == 'June']
juillet = courses_map[courses_map['Mois'] == 'July']
aout = courses_map[courses_map['Mois'] == 'August']
septembre = courses_map[courses_map['Mois'] == 'September']
octobre = courses_map[courses_map['Mois'] == 'October']
novembre = courses_map[courses_map['Mois'] == 'November']
decembre = courses_map[courses_map['Mois'] == 'December']





In [23]:
# create a map for each month :
for month in [mars, avril, mai, juin, juillet, aout, septembre, octobre, novembre, decembre]:
    if month.empty:
        continue

    # create a map with the courses location and marker color by category
    map_courses = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')



    ##### MARKERS #####
    # color by category : blue,       green,    orange,   purple,       red : 
    #                   Gentlemen,   Grimpée,   Chrono,   Access,   Cyclosportive


    # define dataframes by category :
    gentlemen = month[month['Catégorie'] == 'Gentlemen']
    grimpée = month[month['Catégorie'] == 'Grimpée chronométrée']
    test = month[month['Catégorie'] == 'Test chronométré']
    access = month[month['Catégorie'] == 'Access hommes']
    cyclosportive = month[month['Catégorie'] == 'Cyclosportive']


    ##### LAYERS #####
    # each layer corespond to a category
    access_group = folium.FeatureGroup(name="Access").add_to(map_courses)
    for i in range(0,len(access)):
        access_group.add_child(folium.Marker([access.iloc[i]['Latitude'], access.iloc[i]['Longitude']], popup=access.iloc[i]['info_course'], icon=folium.Icon(color='purple', prefix='fa', icon='bicycle')))
    gentlemen_group = folium.FeatureGroup(name="Gentlemen").add_to(map_courses)
    for i in range(0,len(gentlemen)):
        gentlemen_group.add_child(folium.Marker([gentlemen.iloc[i]['Latitude'], gentlemen.iloc[i]['Longitude']], popup=gentlemen.iloc[i]['info_course'], icon=folium.Icon(color='blue', prefix='fa', icon='bicycle')))
    grimpée_group = folium.FeatureGroup(name="Grimpée").add_to(map_courses)
    for i in range(0,len(grimpée)):
        grimpée_group.add_child(folium.Marker([grimpée.iloc[i]['Latitude'], grimpée.iloc[i]['Longitude']], popup=grimpée.iloc[i]['info_course'], icon=folium.Icon(color='green', prefix='fa', icon='bicycle')))
    test_group = folium.FeatureGroup(name="Test chronométré").add_to(map_courses)
    for i in range(0,len(test)):
        test_group.add_child(folium.Marker([test.iloc[i]['Latitude'], test.iloc[i]['Longitude']], popup=test.iloc[i]['info_course'], icon=folium.Icon(color='orange', prefix='fa', icon='bicycle')))
    cyclosportive_group = folium.FeatureGroup(name="Cyclosportive").add_to(map_courses)
    for i in range(0,len(cyclosportive)):
        cyclosportive_group.add_child(folium.Marker([cyclosportive.iloc[i]['Latitude'], cyclosportive.iloc[i]['Longitude']], popup=cyclosportive.iloc[i]['info_course'], icon=folium.Icon(color='red', prefix='fa', icon='bicycle')))

    # add button to switch between OPENSTREETMAP and STAMEN TERRAIN
    folium.TileLayer('OpenStreetMap').add_to(map_courses)
    folium.TileLayer('Stamen Terrain').add_to(map_courses)
    # add a layer control
    folium.LayerControl().add_to(map_courses)




    ##### LEGEND #####
    # add a legend outside the map
    legend_html =   '''
                    <div style="position: fixed;
                                background-color:white; opacity:0.75;
                                bottom: 20px; right: 20px; width: 180px; height: 200px;
                                border:2px solid grey; z-index:9999; font-size:14px;
                                ">&nbsp; Légende : <br>
                                    &nbsp; Gentlemen &nbsp; <i class="fa fa-bicycle fa-2x" style="color:blue"></i><br>
                                    &nbsp; Grimpée &nbsp; <i class="fa fa-bicycle fa-2x" style="color:green"></i><br>
                                    &nbsp; Test chronométré &nbsp; <i class="fa fa-bicycle fa-2x" style="color:orange"></i><br>
                                    &nbsp; Access &nbsp; <i class="fa fa-bicycle fa-2x" style="color:purple"></i><br>
                                    &nbsp; Cyclosportive &nbsp; <i class="fa fa-bicycle fa-2x" style="color:red"></i>
                    </div>
                    '''
    # decrease the size of the legend
    legend_html = legend_html.replace('font-size:1px', 'font-size:6px')
    # write 'Légende' in bold
    legend_html = legend_html.replace('Légende :', '<b>Légende :</b>')
    map_courses.get_root().html.add_child(folium.Element(legend_html))







    # save the map
    map_courses.save('map_courses_' + month.iloc[0]['Mois'] + '.html')

    map_courses
